<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [ ]:
# Load the dataset
data = pd.read_csv('incidents_labelled.csv')

# Combine 'title' and 'text' columns for richer input data
data['combined_text'] = data['title'] + " " + data['text']

# Encode hazard-category to numerical labels
label_mapping = {label: idx for idx, label in enumerate(data['hazard-category'].unique())}
data['label'] = data['hazard-category'].map(label_mapping)

# Stratified train-test split
train_df, val_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

# Define tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))


In [26]:
# Custom Dataset Class for PyTorch
class HazardDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create DataLoaders
train_dataset = HazardDataset(train_df['combined_text'].tolist(), train_df['label'].tolist(), tokenizer)
val_dataset = HazardDataset(val_df['combined_text'].tolist(), val_df['label'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * 3  # Assuming 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


In [28]:
# Training function
def train_model(model, train_loader, val_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_train_loss = 0
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            model.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {avg_train_loss}")

        # Validation after each epoch
        evaluate_model(model, val_loader)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten()

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print("Validation Classification Report:")
    print(classification_report(true_labels, predictions, target_names=label_mapping.keys()))


In [29]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Train and evaluate the model
train_model(model, train_loader, val_loader, optimizer, scheduler)

100%|██████████| 300/300 [02:12<00:00,  2.26it/s]


Epoch 1/3 - Training Loss: 0.7295878298332293
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.87      0.94      0.90       404
                foreign bodies       0.90      0.84      0.87       154
                      chemical       0.82      0.88      0.85       100
                         fraud       0.65      0.55      0.60        82
          organoleptic aspects       1.00      0.23      0.38        13
                     allergens       0.91      0.96      0.94       391
              packaging defect       1.00      0.06      0.12        16
                  other hazard       0.33      0.31      0.32        29
food additives and flavourings       0.00      0.00      0.00         5
                     migration       0.00      0.00      0.00         3

                      accuracy                           0.86      1197
                     macro avg       0.65      0.48   

100%|██████████| 300/300 [02:14<00:00,  2.24it/s]


Epoch 2/3 - Training Loss: 0.3201268105643491
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.95      0.95      0.95       404
                foreign bodies       0.83      0.94      0.88       154
                      chemical       0.84      0.91      0.88       100
                         fraud       0.85      0.65      0.74        82
          organoleptic aspects       1.00      0.54      0.70        13
                     allergens       0.92      0.96      0.94       391
              packaging defect       0.75      0.38      0.50        16
                  other hazard       0.76      0.66      0.70        29
food additives and flavourings       0.00      0.00      0.00         5
                     migration       0.00      0.00      0.00         3

                      accuracy                           0.90      1197
                     macro avg       0.69      0.60   

100%|██████████| 300/300 [02:15<00:00,  2.21it/s]


Epoch 3/3 - Training Loss: 0.18963559216819703
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.93      0.96      0.95       404
                foreign bodies       0.90      0.90      0.90       154
                      chemical       0.84      0.90      0.87       100
                         fraud       0.83      0.66      0.73        82
          organoleptic aspects       0.88      0.54      0.67        13
                     allergens       0.92      0.96      0.94       391
              packaging defect       0.88      0.44      0.58        16
                  other hazard       0.63      0.66      0.64        29
food additives and flavourings       1.00      0.20      0.33         5
                     migration       0.00      0.00      0.00         3

                      accuracy                           0.90      1197
                     macro avg       0.78      0.62  